In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import warnings
import json

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 1000)
pd.set_option('display.max_rows', None) 

# GIDA VE ALKOLSÜZ İÇECEKLER

In [ ]:
gida = {
    "Taze Sebzeler": {
        "Carliston Biber": "https://www.onurmarket.com/biber-carliston-kg--8101",
        "Soğan": "https://www.onurmarket.com/sogan-kuru-dokme-kg--8102",
        "Salatalık": "https://www.onurmarket.com/salatalik-kg--8140",
        "Patlıcan": "https://www.onurmarket.com/patlican-kemer-kg",
        "Taze Fasulye": "https://www.onurmarket.com/-m-fasulye-borulce-kg--8044",
        "Limon": "https://www.onurmarket.com/limon-kg--7965",
        "Maydanoz": "https://www.onurmarket.com/maydanoz-adet--8043"
    },
    "Meyveler": {
        "Domates": "https://www.onurmarket.com/domates-kg--8126",
        "Elma": "https://www.onurmarket.com/elma-starking-kg--7896",
        "Muz": "https://www.onurmarket.com/ithal-muz-kg",
        "Üzüm": "https://www.onurmarket.com/uzum-muskule-kg--7878",
        "Armut": "https://www.onurmarket.com/armut-santa-maria-kg--7997"
    },
    "Ekmek": {
        "Tost Ekmeği": "https://www.onurmarket.com/untad-premium-18-dilim-tost-ekmegi-440-gr-",
        "Tam Buğday Ekmek": "https://www.onurmarket.com/-x.untad-kepek-ekmek-500-gr--48750"
    },
    "Kırmızı Et": {
        "Dana Eti": "https://www.onurmarket.com/-ksp.et-dana-antrikot-kg--121"
    },
    "Diğer Fırıncılık Ürünleri": {
        "Baklava": "https://www.onurmarket.com/seyidoglu-helva-sade-baklava-dilimli-kg-113817",
    },
    "Beyaz Et": {
        "Tavuk Eti": "https://www.onurmarket.com/butun-pilic-kg"
    },
    "Sıvı Yağlar": {
        "Zeytin Yağı": "https://www.onurmarket.com/-komili-sizma-soguk-sikma-500-ml--21344",
        "Ayçiçek Yağı": "https://www.onurmarket.com/-komili-aycicek-pet-4-lt--69469"
    },
    "Peynir": {
        "Kaşar Peyniri": "https://www.onurmarket.com/-ekici-taze-kasar-peyniri-600-gr--70716",
        "Beyaz Peynir": "https://www.onurmarket.com/-icim-peynir-t.yagli-900-gr--60239"
    },
    "Konserve Edilmiş Ürünler": {
        "Salça": "https://www.onurmarket.com/-tat-salca-cam-710-gr--7612",
        "Turşu": "https://www.onurmarket.com/-kuhne-tursu-kornison-670gr-tuzu-az--76068"
    },
    "Diğer": {
        "Yumurta": "https://www.onurmarket.com/onur-bereket-yumurta-30lu-53-63-gr-115742",
        "Çay": "https://www.onurmarket.com/-caykur-tiryaki-1000-gr--3947",
        "Süt": "https://www.onurmarket.com/pinar-sut-25-yagli-1-lt-115056"
    },
    "Patates": {
        "Patates": "https://www.onurmarket.com/patates-kg--8095"
    }
}

data_gida = []
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

for kategori, urunler in gida.items():
    for urun_adi, url in urunler.items():
        try:
            sayfa = requests.get(url, headers=headers, timeout=30)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            urun_isim_tag = html_sayfa.find("div", class_="ProductName")
            if urun_isim_tag:
                urun_isim = urun_isim_tag.find("h1").get_text(strip=True)
            else:
                urun_isim = urun_adi

            fiyat_tag = html_sayfa.find("span", class_="spanFiyat")
            if fiyat_tag:
                fiyat_str = fiyat_tag.get_text(strip=True)
                fiyat = fiyat_str.replace('₺', '').replace('.', '').replace(',', '.').strip()
            else:
                fiyat = None

            data_gida.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': float(fiyat) if fiyat else np.nan
            })

        except requests.RequestException as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_gida = pd.DataFrame(data_gida)

print(df_gida)

In [ ]:
#df_gida

# ALKOLLÜ İÇECEKLER VE TÜTÜN

## SİGARA

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_sigara = {
    'Kategori': 'Sigara',
    'Ürün İsmi': ['Marlboro Touch Blue', 'Parliament Aqua Blue Slims', 'Kent Switch', 'Winston Slender Long'],
    f'Fiyat ({bugun_tarih})': [100.00, 105.00, 97.00, 95.00]
}

df_sigara = pd.DataFrame(data_sigara)

df_sigara[f'Fiyat ({bugun_tarih})'] = df_sigara[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")

## ALKOL

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_bira = {
    'Kategori': 'Alkol',
    'Ürün İsmi': ['50’lik Efes Pilsen Kutu', '50’lik Tuborg Gold Kutu', '50’lik Carlsberg Şişe', '50’lik Bud Şişe'],
    f'Fiyat ({bugun_tarih})': [95.00, 95.00, 98.00, 100.00]
}

df_bira = pd.DataFrame(data_bira)

df_bira[f'Fiyat ({bugun_tarih})'] = df_bira[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")

In [ ]:
df_sigara_alkol = pd.concat([df_sigara, df_bira], ignore_index=True)
df_sigara_alkol

# GİYİM VE AYAKKABI

## GİYİM

In [ ]:
giyim = {
    "Erkek Gİyim": {
        "Gömlek1": "https://www.koton.com/pamuklu-slim-fit-uzun-kollu-italyan-yaka-gomlek-lacivert-4022961-2/",
        "Gömlek2": "https://www.koton.com/pamuk-karisimli-klasik-regular-fit-uzun-kollu-dugmeli-yarim-italyan-yaka-gomlek-mavi-4027614/",
        "Gömlek3": "https://www.koton.com/pamuk-karisimli-klasik-regular-fit-uzun-kollu-dugmeli-yarim-italyan-yaka-gomlek-beyaz-4027613/",
        "Pantolon1": "https://www.koton.com/straight-fit-kot-pantolon-mark-jean-siyah-3956949/",
        "Pantolon2": "https://www.koton.com/pamuklu-dugmeli-slim-fit-jean-pantolon-brad-jean-indigo-3969839-2/",
        "Pantolon3": "https://www.koton.com/dugmeli-pamuk-karisimli-cep-detayli-kumas-pantolon-gri-4019961/"
    },
    "Kadın Giyim": {
        "Gömlek1": "https://www.koton.com/kemer-detayli-dik-yaka-midi-boy-kolsuz-elbise-siyah-3965398-1/",
        "Gömlek2": "https://www.koton.com/kolsuz-mini-polo-yaka-elbise-siyah-4030986/",
        "Gömlek3": "https://www.koton.com/dugmeli-kusakli-cepli-kolsuz-midi-boy-gomlek-elbise-lacivert-4018384/",
        "Pantolon1": "https://www.koton.com/yuksek-bel-cigarette-pantolon-cepli-slim-fit-3935102/",
        "Pantolon2": "https://www.koton.com/yuksek-bel-slim-fit-yipratilmis-ispanyol-paca-denim-pantolon-flare-jeans-dark-indigo-3965821-2/",
        "Pantolon3": "https://www.koton.com/yuksek-bel-crop-duz-paca-mom-fit-denim-pantolon-mom-jean-dark-indigo-4022625/"
    },
    "Çocuk Giyim": {
        "Tişört1": "https://www.koton.com/orumcek-adam-baskili-kisa-kollu-tisort-lisansli-bisiklet-yaka-antrasit-3693127-5/",
        "Tişört2": "https://www.koton.com/pamuklu-kalp-baskili-bisiklet-yaka-kisa-kollu-tisort-gul-4027359-1/",
        "Tişört3": "https://www.koton.com/dinozor-baskili-kisa-kollu-tisort-bisiklet-yaka-ekru-desenli-3695845-2/",
        "Takım1": "https://www.koton.com/pamuklu-uzun-kollu-bisiklet-yaka-sardonlu-basic-esofman-takimi-3931351/",
        "Takım2": "https://www.koton.com/2-li-esofman-takimi-sweatshirt-ve-esofman-alti-sardonlu-mavi-3942261/",
        "Takım3": "https://www.koton.com/2-li-sort-sweatshirt-baski-detayli-pamuklu-ekru-3873555/"
    }
}

data_giyim = []

bugun_tarih = datetime.today().strftime('%Y-%m-%d')

for kategori, urunler in giyim.items():
    for urun_adi, url in urunler.items():
        try:
            sayfa = requests.get(url, timeout=30)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            urun_isim = html_sayfa.find("h1", class_="product-info__header-title")
            urun_isim = urun_isim.get_text(strip=True) if urun_isim else urun_adi
            
            fiyat = html_sayfa.find("div", class_="price__price") 
            fiyat = fiyat.get_text(strip=True) if fiyat else "Fiyat Bulunamadı"

            data_giyim.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': fiyat
            })
        
        except requests.RequestException as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_giyim = pd.DataFrame(data_giyim)

df_giyim[f'Fiyat ({bugun_tarih})'] = df_giyim[f'Fiyat ({bugun_tarih})'] + ' ' 

## AYAKKABI

In [ ]:
ayakkabi = {
    "Erkek Ayakkabı": {
        "Ayakkabı1": "https://www.flo.com.tr/urun/inci-acel-4fx-kahverengi-erkek-klasik-ayakkabi-101544485",
        "Ayakkabı2": "https://www.flo.com.tr/urun/adidas-erkek-spor-ayakkabi-id7110-201257192",
        "Ayakkabı3": "https://www.flo.com.tr/urun/nike-air-monarch-iv-beyaz-erkek-sneaker-100320223"
    },
    "Kadın Ayakkabı": {
        "Ayakkabı1": "https://www.flo.com.tr/urun/inci-tulsi-5fx-siyah-kadin-topuklu-ayakkabi-101973694",
        "Ayakkabı2": "https://www.flo.com.tr/urun/hoops-30-bold-w-201279162",
        "Ayakkabı3": "https://www.flo.com.tr/urun/nike-court-borough-low-recraft-beyaz-unisex-sneaker-101792825"
    },
    "Çocuk Ayakkabı": {
        "Ayakkabı1": "https://www.flo.com.tr/urun/nike-nike-star-runner-4-nn-ps-siyah-erkek-cocuk-kosu-ayakkabisi-101792864",
        "Ayakkabı2": "https://www.flo.com.tr/urun/adidas-runfalcon-5-el-c-lila-kiz-cocuk-spor-ayakkabi-102230997",
        "Ayakkabı3": "https://www.flo.com.tr/urun/reebok-weebok-clasp-low-gri-ub-sneaker-101423866"
    }
}

data_ayakkabi = []

bugun_tarih = datetime.today().strftime('%Y-%m-%d')

for kategori, urunler in ayakkabi.items():
    for urun_adi, url in urunler.items():
        try:
            sayfa = requests.get(url, timeout=10)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            urun_isim = html_sayfa.find("span", class_="js-product-name")
            urun_isim = urun_isim.get_text(strip=True) if urun_isim else urun_adi
            
            fiyat = html_sayfa.find("div", class_="product-pricing-one__price") 
            fiyat = fiyat.get_text(strip=True) if fiyat else "Fiyat Bulunamadı"

            data_ayakkabi.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': fiyat
            })
        
        except requests.RequestException as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_ayakkabi = pd.DataFrame(data_ayakkabi)

df_ayakkabi[f'Fiyat ({bugun_tarih})'] = df_ayakkabi[f'Fiyat ({bugun_tarih})'].str.replace(' TL', ' ', regex=False)  
df_ayakkabi[f'Fiyat ({bugun_tarih})'] = df_ayakkabi[f'Fiyat ({bugun_tarih})'].str.replace('.', ',', regex=False)  
df_ayakkabi[f'Fiyat ({bugun_tarih})'] = df_ayakkabi[f'Fiyat ({bugun_tarih})'].str.replace(',', '.', regex=False)
df_ayakkabi[f'Fiyat ({bugun_tarih})'] = df_ayakkabi[f'Fiyat ({bugun_tarih})'].apply(lambda x: x.replace('.', ',', 1))

In [ ]:
df_giyim_ayakkabi = pd.concat([df_giyim, df_ayakkabi], ignore_index=True)
df_giyim_ayakkabi

# EV EŞYASI

## EV İLE İLGİLİ TEMİZLİK MALZEMELERİ

In [ ]:
temizlik = {
    "Çamaşır Deterjanı": {
        "Çamaşır Deterjanı1": "https://www.onurmarket.com/omo-sivi-26-yikama-active-fresh-1690-ml",
        "Çamaşır Deterjanı2": "https://www.onurmarket.com/-x.ariel-matik-7kg-renklilere-ozel-dag--73518",
        "Çamaşır Deterjanı3": "https://www.onurmarket.com/omo-active-oxygen-toz-camasir-deterjani-beyazlar-icin-6-kg-110817"
    },
    "Yumuşatıcı": {
        "Yumuşatıcı1": "https://www.onurmarket.com/-vernel-max-yumusatici-1440-ml-taze-gul--76639",
        "Yumuşatıcı2": "https://www.onurmarket.com/-bingo-yumusatici-3000-ml-sensitive--18402",
        "Yumuşatıcı3": "https://www.onurmarket.com/-yumos-ext.yumusatici-1440-ml-lilyum--18260"
    },
    "Bulaşık Deterjanı": {
        "Bulaşık Deterjanı1": "https://www.onurmarket.com/-fairy-bulasik-sivisi-650-ml-limon--75994",
        "Bulaşık Deterjanı2": "https://www.onurmarket.com/-pril-bulasik-sivisi-675-gr-limon--75487",
        "Bulaşık Deterjanı3": "https://www.onurmarket.com/finish-quantum-ultimate-48li-bulasik-makinesi-deterjani-tableti-113435"
    }
}

data_temizlik = []
bugun_tarih = datetime.today().strftime('%Y-%m-%d')
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

for kategori, urunler in temizlik.items():
    for urun_adi, url in urunler.items():
        try:
            sayfa = requests.get(url, headers=headers, timeout=10)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            # Ürün adı
            urun_isim_tag = html_sayfa.select_one("div.ProductName h1 span")
            urun_isim = urun_isim_tag.get_text(strip=True) if urun_isim_tag else urun_adi

            # Fiyat bilgisi
            fiyat_tag = html_sayfa.find("span", class_="spanFiyat")
            if fiyat_tag:
                fiyat = fiyat_tag.get_text(strip=True).replace("₺", "").replace(",", ".")
            else:
                fiyat = "Fiyat Bulunamadı"

            data_temizlik.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': fiyat
            })

        except requests.RequestException as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_temizlik = pd.DataFrame(data_temizlik)
df_temizlik[f'Fiyat ({bugun_tarih})'] = pd.to_numeric(df_temizlik[f'Fiyat ({bugun_tarih})'], errors='coerce')
df_temizlik

## MOBİLYA

In [ ]:
mobilya = {
    "Yemek Takımı": {
        "Yemek Takımı1": "https://www.istikbal.com.tr/urun/briella-yemek-odasi-takimi",
        "Yemek Takımı2": "https://www.istikbal.com.tr/urun/alexa-yemek-odasi-takimi",
        "Yemek Takımı3": "https://www.istikbal.com.tr/urun/mitra-yemek-odasi-takimi"
    },
    "Koltuk Takımı": {
        "Koltuk Takımı1": "https://www.istikbal.com.tr/urun/briella-koltuk-takimi",
        "Koltuk Takımı2": "https://www.istikbal.com.tr/urun/alexa-koltuk-takimi",
        "Koltuk Takımı3": "https://www.istikbal.com.tr/urun/mitra-koltuk-takimi-bej"
    },
    "Yatak Odası": {
        "Yatak Odası1": "https://www.istikbal.com.tr/urun/briella-yatak-odasi-takimi",
        "Yatak Odası2": "https://www.istikbal.com.tr/urun/alexa-yatak-odasi-takimi",
        "Yatak Odası3": "https://www.istikbal.com.tr/urun/mitra-yatak-odasi-takimi"
    }
   
}

data_mobilya = []

bugun_tarih = datetime.today().strftime('%Y-%m-%d')

for kategori, urunler in mobilya.items():
    for urun_adi, url in urunler.items():
        try:
            sayfa = requests.get(url, timeout=10)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            urun_isim = html_sayfa.find("div", class_="product-title")
            urun_isim = urun_isim.get_text(strip=True) if urun_isim else urun_adi
            
            fiyat = html_sayfa.find("div", class_="product-price-new")
            fiyat = fiyat.get_text(strip=True) if fiyat else "Fiyat Bulunamadı"

            data_mobilya.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': fiyat
            })
        
        except requests.RequestException as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_mobilya = pd.DataFrame(data_mobilya)

df_mobilya[f'Fiyat ({bugun_tarih})'] = df_mobilya[f'Fiyat ({bugun_tarih})'].str.replace(' TL', ' ', regex=False)  
df_mobilya[f'Fiyat ({bugun_tarih})'] = df_mobilya[f'Fiyat ({bugun_tarih})'].str.replace('.', ',', regex=False)  
df_mobilya[f'Fiyat ({bugun_tarih})'] = df_mobilya[f'Fiyat ({bugun_tarih})'].str.replace(',', '.', regex=False)
df_mobilya[f'Fiyat ({bugun_tarih})'] = df_mobilya[f'Fiyat ({bugun_tarih})'].apply(lambda x: x.replace('.', ',', 1))

## Beyaz Eşya

In [ ]:
urunler = {
    "Buzdolabı": {
        "Buzdolabı1": "https://www.arcelik.com.tr/statik-buzdolabi/d-154140-mb-buzdolabi",
        "Buzdolabı2": "https://www.arcelik.com.tr/alttan-donduruculu-buzdolabi/260316-mb-buzdolabi",
        "Buzdolabı3": "https://www.arcelik.com.tr/ustten-donduruculu-buzdolabi/570475-ei-buzdolabi"
    },
    "Çamaşır Makinesi": {
        "Çamaşır Makinesi1": "https://www.arcelik.com.tr/10-kg-camasir-makinesi/10140-pmf-camasir-makinesi",
        "Çamaşır Makinesi2": "https://www.arcelik.com.tr/10-kg-camasir-makinesi/10140-mp-o-camasir-makinesi",
        "Çamaşır Makinesi3": "https://www.arcelik.com.tr/9-kg-camasir-makinesi/9120-mp-o-camasir-makinesi"
    },
    "Bulaşık Makinesi": {
        "Bulaşık Makinesi1": "https://www.arcelik.com.tr/8-programli-bulasik-makinesi/6688-di-bulasik-makinesi",
        "Bulaşık Makinesi2": "https://www.arcelik.com.tr/3-programli-bulasik-makinesi/6433-bulasik-makinesi",
        "Bulaşık Makinesi3": "https://www.arcelik.com.tr/4-programli-bulasik-makinesi/6444-ok-a-bulasik-makinesi"
    }
}

bugun_tarih = datetime.today().strftime('%Y-%m-%d')
data_beyaz_esya = []

for kategori, urun_dict in urunler.items():
    for urun_adi, url in urun_dict.items():
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
            }

            sayfa = requests.get(url, headers=headers, timeout=10)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            # JSON-LD script'ini bul
            json_ld = html_sayfa.find("script", type="application/ld+json")
            if json_ld:
                json_data = json.loads(json_ld.string)

                urun_isim = json_data.get("name", urun_adi)
                fiyat = json_data.get("offers", {}).get("price", "Fiyat Bulunamadı")
                fiyat += " TL" if fiyat != "Fiyat Bulunamadı" else ""

                data_beyaz_esya.append({
                    "Kategori": kategori,
                    "Ürün İsmi": urun_isim,
                    f'Fiyat ({bugun_tarih})': fiyat
                })
            else:
                print(f"JSON-LD bulunamadı: {urun_adi}")

        except Exception as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_beyaz_esya = pd.DataFrame(data_beyaz_esya)
df_beyaz_esya


In [ ]:
df_ev_esyasi = pd.concat([df_temizlik, df_mobilya, df_beyaz_esya], ignore_index=True)
df_ev_esyasi

# SAĞLIK

## İLAÇ

In [ ]:
ilac = {
    "İlaç": {
        "Aspirin": "https://selcukecza.com.tr/sayfa/ilac-fiyat-ve-barkod-arama/ilac-ve-barkod-arama?barcode=8699546010011&firma=0&ilacname=&kdv=0&gun=01&ay=01&yil=2023",
        "Mannitol": "https://selcukecza.com.tr/sayfa/ilac-fiyat-ve-barkod-arama/ilac-ve-barkod-arama?barcode=8699606691587&firma=0&ilacname=&kdv=0&gun=01&ay=01&yil=2023",
        "Plaquenil": "https://selcukecza.com.tr/sayfa/ilac-fiyat-ve-barkod-arama/ilac-ve-barkod-arama?barcode=8699809098688&firma=0&ilacname=&kdv=0&gun=01&ay=01&yil=2023",
        "Enfluvir": "https://selcukecza.com.tr/sayfa/ilac-fiyat-ve-barkod-arama/ilac-ve-barkod-arama?barcode=8699717150027&firma=0&ilacname=&kdv=0&gun=01&ay=01&yil=2023"
    }
}

data_ilac = []

bugun_tarih = datetime.today().strftime('%Y-%m-%d')

for kategori, urunler in ilac.items():
    for urun_adi, url in urunler.items():
        try:
            sayfa = requests.get(url, timeout=10)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")
            
            satirlar = html_sayfa.find_all("tr")
            
            for satir in satirlar[1:]:
                hucreler = satir.find_all("td")

                if len(hucreler) > 0:
                    barkod = hucreler[0].get_text(strip=True)
                    urun_isim = hucreler[1].get_text(strip=True)
                    fiyat = hucreler[2].get_text(strip=True)

            data_ilac.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': fiyat
            })
        
        except requests.RequestException as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_ilac = pd.DataFrame(data_ilac)

df_ilac[f'Fiyat ({bugun_tarih})'] = df_ilac[f'Fiyat ({bugun_tarih})'].astype(float)
df_ilac[f'Fiyat ({bugun_tarih})'] = df_ilac[f'Fiyat ({bugun_tarih})'].apply(lambda x: "{:,.2f}".format(x))

In [ ]:
df_ilac

# ULAŞTIRMA

## ARAÇ

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_arac = {
    'Kategori': 'Araç',
    'Ürün İsmi': ['Hyundai i20', 'Hyundai Bayon', 'Hyundai Kona', 'Renault Clio','Renault Megane','Renault Captur','Toyota Corolla','Toyota Yaris','Toyota C-HR','Volkswagen Polo','Volkswagen Golf','Volkswagen Tiguan'],
    f'Fiyat ({bugun_tarih})': [1256000.00, 1398000.00, 2161000.00, 1536000.00, 1902900.00, 1900000.00, 1953000.00, 2124000.00, 2276000.00, 1765000.00, 2022000.00, 3049000.00]
}

df_arac = pd.DataFrame(data_arac)

df_arac[f'Fiyat ({bugun_tarih})'] = df_arac[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_arac

## YAKIT

In [ ]:
yakit_turleri = {
    0: "Benzin",       
    1: "Motorin",      
    2: "Gaz"           
}

data_yakit = []
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

url = "https://www.petrolofisi.com.tr/akaryakit-fiyatlari"

try:
    sayfa = requests.get(url, timeout=10)
    sayfa.raise_for_status()
    html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

    fiyat_satiri = html_sayfa.find("tr", class_="price-row district-03431")
    if fiyat_satiri:
        td_liste = fiyat_satiri.find_all("td")[1:]

        for i, td in enumerate(td_liste):
            with_tax_span = td.find("span", class_="with-tax")
            fiyat = with_tax_span.get_text(strip=True) if with_tax_span else "0"
            yakit_turu = yakit_turleri.get(i, f"Yakıt-{i}")

            data_yakit.append({
                "Kategori": "Yakıt",
                "Ürün İsmi": yakit_turu,
                f'Fiyat ({bugun_tarih})': fiyat.replace(",", ".")
            })

except requests.RequestException as e:
    print(f"Hata: Veri çekilemedi - {e}")

df_yakit = pd.DataFrame(data_yakit)

df_yakit[f'Fiyat ({bugun_tarih})'] = df_yakit[f'Fiyat ({bugun_tarih})'].astype(float)
df_yakit[f'Fiyat ({bugun_tarih})'] = df_yakit[f'Fiyat ({bugun_tarih})'].apply(lambda x: "{:,.2f}".format(x))

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

yakit_turleri = {
    0: "Benzin",       # V/Max Kurşunsuz 95
    1: "Motorin",      # V/Max Diesel
    2: "Gaz"           # PO/gaz Otogaz
}

data_yakit = []
bugun_tarih = datetime.today().strftime('%Y-%m-%d')


try:
    sayfa = requests.get(url, timeout=10)
    sayfa.raise_for_status()
    html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

    fiyat_satiri = html_sayfa.find("tr", class_="price-row district-03431")
    if fiyat_satiri:
        td_liste = fiyat_satiri.find_all("td")[1:]  # İlk <td> lokasyon adı, diğerleri fiyatlar

        for i, td in enumerate(td_liste):
            with_tax_span = td.find("span", class_="with-tax")
            fiyat = with_tax_span.get_text(strip=True) if with_tax_span else "0"
            yakit_turu = yakit_turleri.get(i, f"Yakıt-{i}")

            data_yakit.append({
                "Kategori": "Yakıt",
                "Ürün İsmi": yakit_turu,
                f'Fiyat ({bugun_tarih})': fiyat.replace(",", ".")
            })

except requests.RequestException as e:
    print(f"Hata: Veri çekilemedi - {e}")

df_yakit = pd.DataFrame(data_yakit)
df_yakit[f'Fiyat ({bugun_tarih})'] = df_yakit[f'Fiyat ({bugun_tarih})'].astype(float)
df_yakit[f'Fiyat ({bugun_tarih})'] = df_yakit[f'Fiyat ({bugun_tarih})'].apply(lambda x: "{:,.2f}".format(x))
df_yakit


## ŞEHİR İÇİ ULAŞIM

In [ ]:
url = "https://www.metro.istanbul/seferdurumlari/biletucretleri"

sayfa = requests.get(url)
sayfa.raise_for_status()
html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_ulasim = []

tab_content = html_sayfa.find_all('div', class_='tab-pane')

for tab in tab_content:
    kategori = tab.find('h4').text.strip() if tab.find('h4') else "Bilinmeyen Kategori"

    fiyat_listesi = tab.find('ul', class_='price2')

    if fiyat_listesi:
        fiyat_items = fiyat_listesi.find_all('li')

        for item in fiyat_items[:2]:
            urun_isim = item.find('span', class_='float-left').text.strip()
            fiyat = item.find('span', class_='float-right').text.strip()

            data_ulasim.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': fiyat
            })

df_ulasim = pd.DataFrame(data_ulasim)

df_ulasim[f'Fiyat ({bugun_tarih})'] = df_ulasim[f'Fiyat ({bugun_tarih})'].str.replace('₺', '', regex=False) 
df_ulasim[f'Fiyat ({bugun_tarih})'] = df_ulasim[f'Fiyat ({bugun_tarih})'].str.replace('.', '', regex=False)  
df_ulasim[f'Fiyat ({bugun_tarih})'] = df_ulasim[f'Fiyat ({bugun_tarih})'].str.replace(',', '', regex=False)  
df_ulasim[f'Fiyat ({bugun_tarih})'] = df_ulasim[f'Fiyat ({bugun_tarih})'].astype(float)
df_ulasim[f'Fiyat ({bugun_tarih})'] = df_ulasim[f'Fiyat ({bugun_tarih})'].apply(lambda x: "{:,.2f}".format(x))

In [ ]:
df_ulastırma = pd.concat([ df_arac, df_yakit, df_ulasim], ignore_index=True)
df_ulastırma

# HABERLEŞME

## TELEFON MOBIL FİYAT

In [ ]:
url_1 = "https://www.enuygun.com/gsm/tarife-karsilastir.html?key=a3a36790-e942d9bf-c708f0b0-c4339238"
url_2 = "https://www.enuygun.com/gsm/tarife-karsilastir.html?key=c91d351c-6d0b0445-b598f8ce-f14dc3d5"

bugun_tarih = datetime.today().strftime('%Y-%m-%d')

def veri_cek(url):
    sayfa = requests.get(url)
    sayfa.raise_for_status()

    html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

    tarifeler = html_sayfa.find_all("div", class_="filter-col filter-col-2")
    fiyatlar = html_sayfa.find_all("span", class_="price")

    tarife_isimleri = []
    fiyatlar_listesi = []

    for tarife in tarifeler[1:]:
        tarife_ismi = tarife.find("ul")
        if tarife_ismi:
            tarife_ismi = tarife_ismi.find_all("li")[1].text.strip()
            tarife_isimleri.append(tarife_ismi)
        else:
            tarife_isimleri.append("Tarife İsmi Bulunamadı")

    for fiyat in fiyatlar[1:]:  
        fiyat_text = fiyat.text.strip()
        fiyatlar_listesi.append(fiyat_text)

    min_length = min(len(tarife_isimleri), len(fiyatlar_listesi))

    tarife_isimleri = tarife_isimleri[:min_length]
    fiyatlar_listesi = fiyatlar_listesi[:min_length]

    df_tarifeler = pd.DataFrame({
        'Kategori': 'Tarifeler',
        'Ürün İsmi': tarife_isimleri,
        f'Fiyat ({bugun_tarih})': fiyatlar_listesi
    })

    if len(df_tarifeler) >= 8:
        return df_tarifeler.iloc[[1, 3, 5, 7]]
    elif len(df_tarifeler) >= 4:
        return df_tarifeler.sample(4, random_state=42)
    else:
        return df_tarifeler

df_tarifeler_1 = veri_cek(url_1)
df_tarifeler_2 = veri_cek(url_2)

df_tarifeler = pd.concat([df_tarifeler_1, df_tarifeler_2], ignore_index=True)

df_tarifeler[f'Fiyat ({bugun_tarih})'] = df_tarifeler[f'Fiyat ({bugun_tarih})'].str.replace('.', '', regex=False)
df_tarifeler[f'Fiyat ({bugun_tarih})'] = df_tarifeler[f'Fiyat ({bugun_tarih})'].astype(float)
df_tarifeler[f'Fiyat ({bugun_tarih})'] = df_tarifeler[f'Fiyat ({bugun_tarih})'].apply(lambda x: "{:,.2f}".format(x))

## TELEFON AKSESUAR

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_aksesuar_cep = {
    'Kategori': 'Aksesuar',
    'Ürün İsmi': ['TigerIphone 13pro Max Komple Hayalet Kırılmaz Cam Ekran Koruyucu', 'SamsungA24 A34 A54 A74 Süper Hızlı Şarj Adaptörü Type-c Ep-ta800n ( Sadece Başlık )', 'AppleUSB-C Şarj Kablosu 1 Metre - MUF72ZM/A', 'SamsungOrijinal Type-C & Type-C Kablo - Siyah', 'Apple20W USB-C Çıkışlı Güç Adaptörü MHJE3TU/A', 'SamsungEb-u3300x 25w 10.000mah Kablosuz Şarj Özellikli Powerbank Gri Türkiye Garantili'],
    f'Fiyat ({bugun_tarih})': [124.90 , 639.00, 779.00, 299.00, 625.00, 2126.00]
}

df_aksesuar_cep = pd.DataFrame(data_aksesuar_cep)

df_aksesuar_cep[f'Fiyat ({bugun_tarih})'] = df_aksesuar_cep[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_aksesuar_cep

In [ ]:
df_haberlesme = pd.concat([df_tarifeler, df_aksesuar_cep], ignore_index=True)
df_haberlesme

# EĞLENCE VE KÜLTÜR

## TELEVİZYON

In [ ]:
urunler = {
    "Televizyon": {
        "TV1": "https://www.arcelik.com.tr/smart-tv/a55-e-795-b-smart-tv",
        "TV2": "https://www.arcelik.com.tr/smart-tv/a55-e-895-a-smart-tv",
        "TV3": "https://www.arcelik.com.tr/smart-tv/a65-qe-988-a-smart-tv",
        "TV4": "https://www.arcelik.com.tr/smart-tv/a75-e-795-b-smart-tv"
    }
}

bugun_tarih = datetime.today().strftime('%Y-%m-%d')
data_televizyon = []

for kategori, urun_dict in urunler.items():
    for urun_adi, url in urun_dict.items():
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
            }

            sayfa = requests.get(url, headers=headers, timeout=10)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            # JSON-LD script'ini bul
            json_ld = html_sayfa.find("script", type="application/ld+json")
            if json_ld:
                json_data = json.loads(json_ld.string)

                urun_isim = json_data.get("name", urun_adi)
                fiyat = json_data.get("offers", {}).get("price", "Fiyat Bulunamadı")
                fiyat += " TL" if fiyat != "Fiyat Bulunamadı" else ""

                data_televizyon.append({
                    "Kategori": kategori,
                    "Ürün İsmi": urun_isim,
                    f'Fiyat ({bugun_tarih})': fiyat
                })
            else:
                print(f"JSON-LD bulunamadı: {urun_adi}")

        except Exception as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")


df_televizyon = pd.DataFrame(data_televizyon)
df_televizyon

## EĞLENCE (SİNEMA)

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_sinema = {
    'Kategori': 'Sinema',
    'Ürün İsmi': ['Tam', 'Öğrenci'],
    f'Fiyat ({bugun_tarih})': [299.00 , 219.00]
}

df_sinema = pd.DataFrame(data_sinema)

df_sinema[f'Fiyat ({bugun_tarih})'] = df_sinema[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_sinema

## EĞLENCE(DÜĞÜN SALONU/NİŞAN)

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_dugun = {
    'Kategori': 'Düğün',
    'Ürün İsmi': ['Yuvam Düğün & Davet Salonu', 'Poolside Davet Bahçeşehir', 'Violet Balo Davet Beylikdüzü', 'Online Müzik Organizasyon'],
    f'Fiyat ({bugun_tarih})': [75000.00 , 375000.00, 125000.00, 10000.00]
}

df_dugun = pd.DataFrame(data_dugun)

df_dugun[f'Fiyat ({bugun_tarih})'] = df_dugun[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_dugun

## KİTAP

In [ ]:
kitap = {
    "Kitap": {
        "Kitap1": "https://www.dr.com.tr/kitap/beyaz-leke-2-ozgurluk/asli-arslan/edebiyat/roman/turk-romani/urunno=0002160226002",
        "Kitap2": "https://www.dr.com.tr/kitap/el-kizi/edebiyat/roman/turkiye-roman/urunno=0000000262140",
        "Kitap3": "https://www.dr.com.tr/kitap/kral-sakir-mikrop-avcilari-1/varol-yasaroglu/cocuk-ve-genclik/okul-cagi-6-10-yas/cocuk-oyku-hikaye/urunno=0001942348001",
        "Kitap4": "https://www.dr.com.tr/kitap/yds-5-plus-tamami-cozumlu-deneme-sinavi-9607/kolektif/egitim-ve-sinav-kitaplari/osym-sinavlarina-hazirlik/yds/yds-deneme-sinavi-yaprak-test/urunno=0001873748001"
    }
}

data_kitap = []

bugun_tarih = datetime.today().strftime('%Y-%m-%d')

for kategori, urunler in kitap.items():
    for urun_adi, url in urunler.items():
        try:
            sayfa = requests.get(url, timeout=10)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            urun_isim = html_sayfa.find("h1", class_="fs-7 mb-0 js-text-prd-name")
            urun_isim = urun_isim.get_text(strip=True) if urun_isim else urun_adi
            
            fiyat = html_sayfa.find("span", class_="current-price js-text-current-price")
            if not fiyat:
                fiyat = html_sayfa.find("span", class_="current-price js-text-current-price product-detail-prd-campaign-price-gray")
            fiyat = fiyat.get_text(strip=True) if fiyat else "Fiyat Bulunamadı"

            data_kitap.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': fiyat
            })
        
        except requests.RequestException as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_kitap = pd.DataFrame(data_kitap)

df_kitap[f'Fiyat ({bugun_tarih})'] = df_kitap[f'Fiyat ({bugun_tarih})'].str.replace('TL', ' ', regex=False) 
df_kitap[f'Fiyat ({bugun_tarih})'] = df_kitap[f'Fiyat ({bugun_tarih})'].str.replace(',', '.', regex=False) 

In [ ]:
df_eglence = pd.concat([df_televizyon, df_sinema,df_dugun, df_kitap], ignore_index=True)
df_eglence

# EĞİTİM

## ANAOKULU-ORTAOKUL-LİSE

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_egitim_1 = {
    'Kategori': 'Eğitim Ücreti',
    'Ürün İsmi': ['Anaokulu', ' İlkokul', 'Ortaokul','Lise','Ortaokul Kurs','Lise Kurs'],
    f'Fiyat ({bugun_tarih})': [432000.00 , 380800.00, 373800.00, 400700.00, 37500.00, 45000.00,]
}

df_egitim_1 = pd.DataFrame(data_egitim_1)

df_egitim_1[f'Fiyat ({bugun_tarih})'] = df_egitim_1[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_egitim_1

## ÜNİVERSİTE

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_universite = {
    'Kategori': 'Üniversite',
    'Ürün İsmi': ['İngilizce Hazırlık Ön Lisans (Peşin)', ' İngilizce Hazırlık Ön Lisans (Taksitli)', 'İngilizce Hazırlık Lisans (Peşin)','İngilizce Hazırlık Lisans (Taksitli)','Denizcilik Meslek Yüksekokulu (Peşin)','Denizcilik Meslek Yüksekokulu (Taksitli)','Denizcilik Fakültesi (Peşin)','Denizcilik Fakültesi (Taksitli)','Mühendislik Fakültesi (Peşin)','Mühendislik Fakültesi (Taksitli)','İktisadi ve İdari Bilimler Fakültesi (Peşin)','İktisadi ve İdari Bilimler Fakültesi (Taksitli)','Hukuk Fakültesi (Peşin)','Hukuk Fakültesi (Taksitli)'],
    f'Fiyat ({bugun_tarih})': [374000.00 , 467500.00, 466400.00,583000.00,374000.00,467500.00,572000.00,715000.00,572000.00,715000.00,572000.00,715000.00,572000.00,715000.00]
}

df_universite = pd.DataFrame(data_universite)

df_universite[f'Fiyat ({bugun_tarih})'] = df_universite[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_universite

In [ ]:
df_egitim = pd.concat([df_egitim_1, df_universite], ignore_index=True)
df_egitim

# LOKANTA VE OTELLER

## YEMEKLER

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_yemek = {
    'Kategori': 'Yemek',
    'Ürün İsmi': ['BK Steakhouse Burger®Patates Kızartması (Orta)Soğan Halkası (6lı)Kutu İçecekx2 Sos', ' Big King®Patates Kızartması (Orta)Kutu İçecekx2 Sos', 'Smoky XL SandviçPatates Kızartması (Orta)Kutu İçecekx2 Sos','x2 Tavukburger®Nuggets (8li)Tenders (2li)Patates Kızartması (Büyük)1 L. İçecekx2 Sos','BBQ Gurme TavukTırtıklı Patates (Orta)Kutu İçecekx2 Sos','Beefn Cheddar XLTırtıklı Patates (Orta)Kutu İçecekx2 Sos','Büyük Boy Sucuklu','Büyük Boy Margarita','Usta Dönerci® Et İskender','5li KöftePilav','Dürüm Et Döner','Pilav','Mercimek Çorbası','Kuşbaşılı Kaşarlı Pide','Lahmacun','Teriyaki Tavuk Sandviç'],
    f'Fiyat ({bugun_tarih})': [400.00 , 310.00, 320.00, 500.00, 270.00,400.00,325.00,315.00,325.00,255.00,295.00,45.00,55.00,290.00,100.00,360.00]
}

df_yemek = pd.DataFrame(data_yemek)

df_yemek[f'Fiyat ({bugun_tarih})'] = df_yemek[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_yemek

## İÇECEKLER

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_icecek = {
    'Kategori': 'İçecek',
    'Ürün İsmi': ['Coca-Cola','Ayran','Fuse Tea'],
    f'Fiyat ({bugun_tarih})': [50.00 , 45.00, 60.00]
}

df_icecek = pd.DataFrame(data_icecek)

df_icecek[f'Fiyat ({bugun_tarih})'] = df_icecek[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_icecek

## OTEL

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

url = "https://yurtburs.itu.edu.tr/yurt/yurt-ucretleri"

sayfa = requests.get(url, timeout=10)
sayfa.raise_for_status()
html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

tablo = html_sayfa.find("table")

if tablo:
    rows = tablo.find_all("tr")

    headers = [th.get_text(strip=True) for th in rows[0].find_all("th")]
    if not headers:
        headers = [td.get_text(strip=True) for td in rows[0].find_all("td")]

    if len(headers) == 1 and "EĞİTİM-ÖĞRETİM YILI" in headers[0]:
        headers = [td.get_text(strip=True) for td in rows[1].find_all("td")]
        data_start_index = 2
    else:
        data_start_index = 1

    data_yurt = []
    for row in rows[data_start_index:]:
        columns = row.find_all("td")
        row_data = [col.get_text(strip=True) for col in columns]
        if row_data:
            data_yurt.append(row_data)

    if len(headers) == 3:
        headers.append('EKSTRA BİLGİ')

    df_yurt = pd.DataFrame(data_yurt, columns=headers)
    df_yurt["Kategori"] = "Yurt Ücreti"
    df_yurt[f'Fiyat ({bugun_tarih})'] = df_yurt.iloc[:, -1]

df_yurt = df_yurt[["Kategori", "YURT ADI", "TEMİNATBEDELİ"]]
df_yurt = df_yurt.iloc[:3] 
df_yurt.rename(columns={'Kategori': 'Kategori', 
                        'YURT ADI': 'Ürün İsmi',
                        'TEMİNATBEDELİ': f'Fiyat ({bugun_tarih})'}, inplace=True)

df_yurt[f'Fiyat ({bugun_tarih})'] = df_yurt[f'Fiyat ({bugun_tarih})'].str.replace('₺', '', regex=False) 
df_yurt[f'Fiyat ({bugun_tarih})'] = df_yurt[f'Fiyat ({bugun_tarih})'].str.replace('.', '', regex=False)  
df_yurt[f'Fiyat ({bugun_tarih})'] = df_yurt[f'Fiyat ({bugun_tarih})'].str.replace(',', '.', regex=False) 
df_yurt[f'Fiyat ({bugun_tarih})'] = df_yurt[f'Fiyat ({bugun_tarih})'].astype(float)
df_yurt[f'Fiyat ({bugun_tarih})'] = df_yurt[f'Fiyat ({bugun_tarih})'].apply(lambda x: "{:,.2f}".format(x))

In [ ]:
# df_lokanta_otel = pd.concat([df_yemek, df_icecek, df_yurt], ignore_index=True)
df_lokanta_otel = pd.concat([df_yemek,df_icecek,df_yurt], ignore_index=True)
df_lokanta_otel

# ÇEŞİTLİ MAL VE HİZMETLER

## KİŞİSEL HİJYEN VE BAKIM

In [ ]:
bakim = {
    "Deodorant": {
        "Deodorant1": "https://www.onurmarket.com/-axe-deo-sprey-150ml-apollo--25028",
        "Deodorant2": "https://www.onurmarket.com/-nivea-roll-on-50ml-bw-clear-women--76616",
        "Deodorant3": "https://www.onurmarket.com/rexona-deodorant-invisible-fresh-sprey-150-ml-111574"
    },
    "Şampuan": {
        "Şampuan1": "https://www.onurmarket.com/elidor-avokado-sampuan-200-ml-bakim-yagi-80-ml",
        "Şampuan2": "https://www.onurmarket.com/-dove-samp.400ml-besleyici-bakim--74319",
        "Şampuan3": "https://www.onurmarket.com/-hs-samp.-350ml-21-klasik-bakim--76506"
    },
    "Tuvalet Kağıdı": {
        "Tuvalet Kağıdı1": "https://www.onurmarket.com/papia-tuvalet-kagidi-24lu-3-katli-116268",
        "Tuvalet Kağıdı2": "https://www.onurmarket.com/-x.selpak-tuvalet-kagidi-16-li--77012",
        "Tuvalet Kağıdı3": "https://www.onurmarket.com/-solo-tuvalet-kagidi-32li--76640"
    },
    "Traş Köpüğü": {
        "Traş Köpüğü1": "https://www.onurmarket.com/-derby-tiras-kopuk-200ml-normal--10198",
        "Traş Köpüğü2": "https://www.onurmarket.com/gillette-tiras-kopugu-hassas-200-ml",
        "Traş Köpüğü3": "https://www.onurmarket.com/-arko-tras-jel-200ml-cool--9969"
    }
}

data_bakim = []
bugun_tarih = datetime.today().strftime('%Y-%m-%d')
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

for kategori, urunler in bakim.items():
    for urun_adi, url in urunler.items():
        try:
            sayfa = requests.get(url, headers=headers, timeout=10)
            sayfa.raise_for_status()
            html_sayfa = BeautifulSoup(sayfa.content, "html.parser")

            # Ürün ismi
            urun_isim_tag = html_sayfa.select_one("div.ProductName h1 span")
            urun_isim = urun_isim_tag.get_text(strip=True) if urun_isim_tag else urun_adi

            # Fiyat bilgisi
            fiyat_tag = html_sayfa.find("span", class_="spanFiyat")
            if fiyat_tag:
                fiyat = fiyat_tag.get_text(strip=True).replace("₺", "").replace(",", ".")
            else:
                fiyat = "Fiyat Bulunamadı"

            data_bakim.append({
                "Kategori": kategori,
                "Ürün İsmi": urun_isim,
                f'Fiyat ({bugun_tarih})': fiyat
            })

        except requests.RequestException as e:
            print(f"Hata: {urun_adi} ürünü için veri çekilemedi: {e}")

df_bakim = pd.DataFrame(data_bakim)

df_bakim[f'Fiyat ({bugun_tarih})'] = pd.to_numeric(df_bakim[f'Fiyat ({bugun_tarih})'], errors='coerce')

df_bakim


## AVUKATLIK ÜCRETİ

In [ ]:
url = "https://www.istanbulbarosu.org.tr/ucrettarifeleri.aspx"

sayfa = requests.get(url)
sayfa.raise_for_status()
html_sayfa = BeautifulSoup(sayfa.content, "html.parser")
satirlar = html_sayfa.find_all("tr")

data_ucret = []
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

for satir in satirlar:
    hucreler = satir.find_all("td")
    
    if len(hucreler) == 3:
        hizmet_adi = hucreler[1].get_text(strip=True)
        fiyat = hucreler[2].get_text(strip=True)
 
        if "Resmî Gazete" not in hizmet_adi and "Sözleşmeler ve benzeri belgelerin hazırlanması" not in hizmet_adi:
            data_ucret.append({
                "Kategori": "Avukatlık Ücreti",
                "Ürün İsmi": hizmet_adi,
                f'Fiyat ({bugun_tarih})': fiyat
            })

df_avukatlik = pd.DataFrame(data_ucret)

df_avukatlik[f'Fiyat ({bugun_tarih})'] = df_avukatlik[f'Fiyat ({bugun_tarih})'].str.replace('TL', '', regex=False) 
df_avukatlik[f'Fiyat ({bugun_tarih})'] = df_avukatlik[f'Fiyat ({bugun_tarih})'].str.replace(',', '.', regex=False) 
df_avukatlik[f'Fiyat ({bugun_tarih})'] = df_avukatlik[f'Fiyat ({bugun_tarih})'].astype(float)
df_avukatlik[f'Fiyat ({bugun_tarih})'] = df_avukatlik[f'Fiyat ({bugun_tarih})'].apply(lambda x: "{:,.2f}".format(x))

## NOTER ÜCRETİ

In [ ]:
url = "https://kadimhukuk.com.tr/makale/noter-ucretleri/"

sayfa = requests.get(url)
sayfa.raise_for_status()
html_sayfa = BeautifulSoup(sayfa.content, "html.parser")
satirlar = html_sayfa.find_all("tr")

data_noter = []
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

for satir in satirlar:
    hucreler = satir.find_all("td")

    if len(hucreler) == 2:
        hizmet_adi = hucreler[0].get_text(strip=True) 
        fiyat = hucreler[1].get_text(strip=True)
        
        fiyat_sadece = re.sub(r"[^0-9,\.]", "", fiyat)

        if "Taşınmaz Satış Sözleşmesi Ücreti" not in hizmet_adi and "Düzenleme Ücreti" not in hizmet_adi and "Defter Onaylama Ücreti" not in hizmet_adi and "Harç ve Vergiden Muaf İşlemler Ücreti" not in hizmet_adi and "Noter ihtarname ücretleri" not in hizmet_adi:
            data_noter.append({
                "Kategori": "Noter Ücreti",
                "Ürün İsmi": hizmet_adi,
                f'Fiyat ({bugun_tarih})': fiyat_sadece
            })

df_noter = pd.DataFrame(data_noter)

df_noter[f'Fiyat ({bugun_tarih})'] = df_noter[f'Fiyat ({bugun_tarih})'].str.replace('.', '', regex=False) 
df_noter[f'Fiyat ({bugun_tarih})'] = df_noter[f'Fiyat ({bugun_tarih})'].str.replace(',', '.', regex=False) 
df_noter[f'Fiyat ({bugun_tarih})'] = df_noter[f'Fiyat ({bugun_tarih})'].astype(float)
df_noter[f'Fiyat ({bugun_tarih})'] = df_noter[f'Fiyat ({bugun_tarih})'].apply(lambda x: "{:,.2f}".format(x))

# Mücevherat,Saat ve Kol Saatleri

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_saat = {
    'Kategori': 'Saat',
    'Ürün İsmi': ['Saat1', 'Saat 2', 'Saat 3'],
    f'Fiyat ({bugun_tarih})': [1799.00 , 13945.00, 4129.00]
}

df_saat = pd.DataFrame(data_saat)

df_saat[f'Fiyat ({bugun_tarih})'] = df_saat[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_saat

In [ ]:
df_cesitli_mal_hizmetler = pd.concat([df_bakim, df_avukatlik, df_noter, df_saat], ignore_index=True)
df_cesitli_mal_hizmetler

# KONUT

## ELEKTRİK

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_elektrik = {
    'Kategori': 'Elektrik',
    'Ürün İsmi': ['Mesken (Düşük Kademe)', 'Mesken (Yüksek Kademe)', 'Ev (Mesken) Düşük Kademe'],
    f'Fiyat ({bugun_tarih})': [2.07 , 3.11, 3.15]
}

df_elektrik = pd.DataFrame(data_elektrik)

df_elektrik[f'Fiyat ({bugun_tarih})'] = df_elektrik[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")
df_elektrik

## SU

In [ ]:
bugun_tarih = datetime.today().strftime('%Y-%m-%d')

url = "https://iski.istanbul/abone-hizmetleri/abone-rehberi/su-birim-fiyatlari/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('tbody')
rows = table.find_all('tr')

data_su = []

for row in rows:
    cols = row.find_all('td')
    if len(cols) >= 4:
        abone_gruplari = cols[0].text.strip()
        toplam_birim_fiyat = cols[3].text.strip()
        data_su.append({
                    "Kategori": "Su",
                    "Ürün İsmi": abone_gruplari,
                    f'Fiyat ({bugun_tarih})': toplam_birim_fiyat
                })

df_su = pd.DataFrame(data_su)
df_su = df_su.iloc[1:4]

df_su[f'Fiyat ({bugun_tarih})'] = df_su[f'Fiyat ({bugun_tarih})'].str.replace(',', '.', regex=False)

## DOĞALGAZ

In [ ]:
url = 'https://igdas.istanbul/perakende-satis/' #KONTROLLER YAPILACAK

bugun_tarih = datetime.today().strftime('%Y-%m-%d')

data_dogalgaz = {
    'Kategori': 'Doğalgaz',
    'Ürün İsmi': ['Abone Doğalgaz Satış Fiyatı'],
    f'Fiyat ({bugun_tarih})': [9.88]
}

df_dogalgaz = pd.DataFrame(data_dogalgaz)

df_dogalgaz[f'Fiyat ({bugun_tarih})'] = df_dogalgaz[f'Fiyat ({bugun_tarih})'].apply(lambda x: f"{x:.2f}")

## GERÇEK KİRA

In [ ]:
df_konut = pd.concat([df_elektrik, df_su, df_dogalgaz], ignore_index=True)
df_konut

# NİHAİ TUFE VERİSİ

In [ ]:
df_tufe =  pd.concat([df_gida, df_sigara_alkol, df_giyim_ayakkabi, df_ev_esyasi, df_ilac, df_ulastırma, df_haberlesme, 
                      df_eglence, df_egitim, df_lokanta_otel, df_cesitli_mal_hizmetler, df_konut], ignore_index=True)

df_tufe

In [ ]:
 bugun_tarih = datetime.today().strftime('%Y-%m-%d')

 df_tufe.to_excel(f"tufe_({bugun_tarih}).xlsx")
